# 5.5.2 Discontinuous Galerkin for the Wave Equation

We solve the first order wave equation by a matrix-free explicit DG method:

\begin{eqnarray*}
\frac{\partial p}{\partial t} & = & \operatorname{div} u \\
\frac{\partial u}{\partial t} & = & \nabla p
\end{eqnarray*}


Using DG discretization in space we obtain the ODE system
\begin{eqnarray*}
M_p \dot{p} & = & -B^T u \\
M_u \dot{u} & = & B p,
\end{eqnarray*}

with mass-matrices $M_p$ and $M_u$, and the discretization matrix $B$ for the gradient, and $-B^T$ for the divergence. 


The DG bilinear-form for the gradient is:

$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\},
$$
where $\{ p \}$ is the average of $p$ on the facet.

The computational advantages of a proper version of DG is:

* universal element-matrices for the differntial operator $B$
* cheaply invertible mass matrices (orthogonal polynomials, sum-factorization)


In [1]:
from ngsolve import *
from netgen.occ import *
from time import time

from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

box = Box((-1,-1,-1), (1,1,0))
sp = Sphere((0.5,0,0), 0.2)
shape = box-sp
geo = OCCGeometry(shape)

h = 0.1
        
mesh = Mesh( geo.GenerateMesh(maxh=h))
mesh.Curve(3)
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [2]:
order = 4
fes_p = L2(mesh, order=order, all_dofs_together=True)
fes_u = VectorL2(mesh, order=order, piola=True)
fes_tr = FacetFESpace(mesh, order=order)

print ("ndof_p = ", fes_p.ndof, "+", fes_tr.ndof, ", ndof_u =", fes_u.ndof)

traceop = fes_p.TraceOperator(fes_tr, average=True) 

gfu = GridFunction(fes_u)
gfp = GridFunction(fes_p)
gftr = GridFunction(fes_tr)

gfp.Interpolate( exp(-400*(x**2+y**2+z**2)))
gftr.vec.data = traceop * gfp.vec

ndof_p =  574175 + 519660 , ndof_u = 1722525


In [3]:
p = fes_p.TrialFunction()
v = fes_u.TestFunction()
phat = fes_tr.TrialFunction()

n = specialcf.normal(mesh.dim)

$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\},
$$

where $\{ p \}$ is the average of $p$ on the facet.


In [4]:
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p)*v * dx -p*(v*n) * dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += phat * (v*n) *dx(element_boundary=True)
Btr.Assemble();

B = Bel.mat + Btr.mat @ traceop

In [5]:
invmassp = fes_p.Mass(1).Inverse()
invmassu = fes_u.Mass(1).Inverse()

In [6]:
gfp.Interpolate(exp(-100*(x**2 + y**2 + z**2)))
gfu.vec[:] = 0

scene = Draw(gftr, draw_vol=False, order=3, min=-0.05, max=0.05)

t = 0.0
dt = 0.3 * h / (order + 1)**2
tend = 1.0

op1 = dt * invmassu @ B
op2 = dt * invmassp @ B.T

cnt = 0
with TaskManager():
    while t < tend:
        t += dt
        gfu.vec.data += op1 * gfp.vec   # u-step
        gfp.vec.data -= op2 * gfu.vec   # p-step
        cnt += 1

        if cnt % 10 == 0:
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()

            nu = float(gfu.vec.Norm())
            np = float(gfp.vec.Norm())
            print(f"[CPU] step={cnt:5d}  t={t:.4f}  ||u||={nu:.3e}  ||p||={np:.3e}")


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

[CPU] step=   10  t=0.0120  ||u||=8.188e-03  ||p||=2.637e+00
[CPU] step=   20  t=0.0240  ||u||=1.575e-02  ||p||=2.431e+00
[CPU] step=   30  t=0.0360  ||u||=2.216e-02  ||p||=2.133e+00
[CPU] step=   40  t=0.0480  ||u||=2.708e-02  ||p||=1.800e+00
[CPU] step=   50  t=0.0600  ||u||=3.036e-02  ||p||=1.505e+00
[CPU] step=   60  t=0.0720  ||u||=3.210e-02  ||p||=1.327e+00
[CPU] step=   70  t=0.0840  ||u||=3.255e-02  ||p||=1.302e+00
[CPU] step=   80  t=0.0960  ||u||=3.210e-02  ||p||=1.391e+00
[CPU] step=   90  t=0.1080  ||u||=3.117e-02  ||p||=1.519e+00
[CPU] step=  100  t=0.1200  ||u||=3.012e-02  ||p||=1.638e+00
[CPU] step=  110  t=0.1320  ||u||=2.918e-02  ||p||=1.725e+00
[CPU] step=  120  t=0.1440  ||u||=2.846e-02  ||p||=1.778e+00
[CPU] step=  130  t=0.1560  ||u||=2.797e-02  ||p||=1.802e+00
[CPU] step=  140  t=0.1680  ||u||=2.766e-02  ||p||=1.806e+00
[CPU] step=  150  t=0.1800  ||u||=2.747e-02  ||p||=1.800e+00
[CPU] step=  160  t=0.1920  ||u||=2.738e-02  ||p||=1.790e+00
[CPU] step=  170  t=0.20

## Time-stepping on the device

In [7]:
#try:
#    import ngsolve.ngscuda
#except:
#    print ("Sorry, no cuda device")

import sys
sys.path.insert(0, "/home/fs72329/ntylek/myngs_cuda")  

import ngsolve
import ngscuda

print("import ok")
print("NGSolve path:", ngsolve.__file__)
print("ngscuda path:", ngscuda.__file__)



CUDA Device Query...
There is 1 CUDA device.
CUDA Device 0: NVIDIA A40, cap 8.6
Using device 0
import ok
NGSolve path: /opt/conda/lib/python3.10/site-packages/ngsolve/__init__.py
ngscuda path: /home/fs72329/ntylek/myngs_cuda/ngscuda.so
Initializing cublas and cusparse.


In [8]:
logfile = open("gpu_output.txt", "w") 

gfp.Interpolate(exp(-100*(x**2 + y**2 + z**2)))
gfu.vec[:] = 0

scene = Draw(gftr, draw_vol=False, order=3, min=-0.05, max=0.05)

t = 0
# dt = 0.3 * h / (order + 1)**2 / 2 
dt = 0.3 * h / (order + 1)**2 
tend = 1

op1 = (dt * invmassu @ B).CreateDeviceMatrix()
op2 = (dt * invmassp @ B.T).CreateDeviceMatrix()

devu = gfu.vec.CreateDeviceVector(copy=True)
devp = gfp.vec.CreateDeviceVector(copy=True)

cnt = 0
with TaskManager():
    while t < tend:
        t += dt
        devu += op1 * devp
        devp -= op2 * devu
        cnt += 1

        if cnt % 10 == 0:
            gfp.vec.data = devp
            gftr.vec.data = traceop * gfp.vec

            scene.Redraw()

            line = f"[GPU] step={cnt:5d}  t={t:.4f}  ||p||={float(gfp.vec.Norm()):.3e}\n"
            logfile.write(line) 
            logfile.flush()    

logfile.close()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [9]:
logfile = open("gpu_output.txt", "w")
def log(line): 
    print(line, flush=True); logfile.write(line+"\n"); logfile.flush()

# ICs
gfp.Interpolate(exp(-100*(x**2 + y**2 + z**2)))
gfu.vec[:] = 0

scene = Draw(gftr, draw_vol=False, order=3, min=-0.05, max=0.05)

t = 0.0
dt = 0.3 * h / (order + 1)**2
tend = 1.0

# device operators
op1 = (dt * invmassu @ B).CreateDeviceMatrix()   # maps p -> Δu
op2 = (dt * invmassp @ B.T).CreateDeviceMatrix() # maps u -> Δp

# device vectors
devu = gfu.vec.CreateDeviceVector(copy=True)
devp = gfp.vec.CreateDeviceVector(copy=True)

# independent temporaries (avoid aliasing)
tmp_u = gfu.vec.CreateDeviceVector(copy=True)
tmp_p = gfp.vec.CreateDeviceVector(copy=True)

# initial norms
gfu.vec.data = devu; nu0 = float(gfu.vec.Norm())
gfp.vec.data = devp; np0 = float(gfp.vec.Norm())
log(f"init ||u||={nu0:.3e} ||p||={np0:.3e}")

cnt = 0
with TaskManager():
    while t < tend:
        t += dt

        # u += op1 * p
        op1.Mult(devp, tmp_u)
        devu += tmp_u

        # p -= op2 * u
        op2.Mult(devu, tmp_p)
        devp -= tmp_p

        cnt += 1
        if cnt % 10 == 0:
            gfu.vec.data = devu; nu = float(gfu.vec.Norm())
            gfp.vec.data = devp; np = float(gfp.vec.Norm())
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()
            log(f"[GPU] step={cnt:5d}  t={t:.4f}  ||u||={nu:.3e}  ||p||={np:.3e}")

logfile.close()


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

NgException: cudaMalloc error, ec=700

In [ ]:
print (op1.GetOperatorInfo())

In [ ]:
ts = time()
steps = 10
for i in range(steps):
    devu += op1 * devp
    devp -= op2 * devu
te = time()
print ("ndof = ", gfp.space.ndof, "+", gfu.space.ndof, ", time per step =", (te-ts)/steps)

On the A-100:

## Time-domain PML

PML (perfectly matched layers) is a method for approximating outgoing waves on a truncated domain. Its time domain version leads to additional field variables in the PML region, which are coupled via time-derivatives only.

In [ ]:
# from ring_resonator_import import *
from ngsolve import *
from ngsolve.webgui import Draw

In [ ]:
gfu = GridFunction(fes)
scene = Draw (gfu.components[0], order=3, min=-0.05, max=0.05, autoscale=False)

t = 0
tend = 15
tau = 2e-4
i = 0
sigma = 10   # pml damping parameter

op1 = invp@(-fullB.T-sigma*dampingp) 
op2 = invu@(fullB-sigma*dampingu)
with TaskManager(): 
    while t < tend:

        gfu.vec.data += tau*Envelope(t)*srcvec
        gfu.vec.data += tau*op1*gfu.vec
        gfu.vec.data += tau*op2*gfu.vec        

        t += tau
        i += 1
        if i%20 == 0:
            scene.Redraw()

The differential operators and coupling terms to the pml - variables are represented via linear operators;

In [ ]:
print (fullB.GetOperatorInfo())
print (dampingp.GetOperatorInfo())

In [ ]:
print (op1.GetOperatorInfo())